In [1]:
import pandas as pd
import numpy as np
import ast
import re
import sklearn
import six
from abc import ABCMeta
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')
import nltk
import six
import re
import gensim

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from abc import ABCMeta
from sklearn.pipeline import Pipeline
from sklearn.utils import check_X_y, check_array
from scipy import sparse
from nltk.stem import PorterStemmer,WordNetLemmatizer
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score

In [2]:
tsel = pd.read_csv(r'Prepro_Pelayanan.csv', usecols=['Username','Text','Pelayanan','text_result'])
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,"['handphone', 'lapor', 'wajib', 'bantu', 'laca..."
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,"['tidak tahu', 'telkomsel', 'lambat', 'kuota',..."
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,"['kendala', 'sinyal', 'telkomsel']"
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,"['tidak bisa', 'nomor', 'mati', 'dihidupakan',..."
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,"['admin', 'kemarin', 'aplikasi', 'punya', 'tel..."
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,"['sinyal', 'minus', 'tolong', 'baik', 'problem..."
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,"['punya', 'telkomsel', 'ganggu', 'aduh', 'desa..."
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,"['allah', 'tambah', 'kerja', 'esuk', 'neng', '..."
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,"['admin', 'sinyal', 'telkomsel', 'rumah', 'dae..."


In [3]:
tsel['text_result'] = tsel['text_result'].apply(str)

In [4]:
import ast

def join_text_list(text):
    texts = ast.literal_eval(text)
    return ' '.join([text for text in texts])

tsel['text_result'] = tsel['text_result'].apply(join_text_list)
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,handphone lapor wajib bantu lacak imei perihal...
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,tidak tahu telkomsel lambat kuota promo murah ...
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,kendala sinyal telkomsel
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,tidak bisa nomor mati dihidupakan grapari nomo...
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,admin kemarin aplikasi punya telkomsel ganggu ...
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,sinyal minus tolong baik problem location cama...
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,punya telkomsel ganggu aduh desak pakai
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,allah tambah kerja esuk neng grapari
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,admin sinyal telkomsel rumah daerah rawabunga ...


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer(max_features=5000)
vectors = matrix.fit_transform(tsel['text_result']).toarray()

In [24]:
X = vectors
y = tsel['Pelayanan']

In [40]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.1, random_state=45)

## Klasifikasi NBSVM

In [41]:
class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):
    def __init__(self, alpha=1.0, C=1.0, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.C = C
        self.svm_ = [] # fuggly

    def fit(self, X, y):
        X, y = check_X_y(X, y, 'csr')
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # so we don't have to cast X to floating point
        Y = Y.astype(np.float64)

        # Count raw events from data
        n_effective_classes = Y.shape[1]
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
                                 dtype=np.float64)
        self._compute_ratios(X, Y)

        # flugglyness
        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            svm = LinearSVC(C=self.C, max_iter=self.max_iter)
            Y_i = Y[:,i]
            svm.fit(X_i, Y_i)
            self.svm_.append(svm) 

        return self

    def predict(self, X):
        n_effective_classes = self.class_count_.shape[0]
        n_examples = X.shape[0]

        D = np.zeros((n_effective_classes, n_examples))

        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            D[i] = self.svm_[i].decision_function(X_i)
        
        return self.classes_[np.argmax(D, axis=0)]
        
    def _compute_ratios(self, X, Y):
        """Count feature occurrences and compute ratios."""
        if np.any((X.data if issparse(X) else X) < 0):
            raise ValueError("Input X must be non-negative")

        self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
        normalize(self.ratios_, norm='l1', axis=1, copy=False)
        row_calc = lambda r: np.log(np.divide(r, (1 - r)))
        self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
        check_array(self.ratios_)
        self.ratios_ = sparse.csr_matrix(self.ratios_)

In [42]:
x_train_csr = sparse.csr_matrix(x_train)
x_test_csr = sparse.csr_matrix(x_test)
model = NBSVM()

model.fit(x_train_csr,y_train)
y_pred = model.predict(x_test_csr)
y_true = y_test

### random 5

In [11]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 121  106   18]
 [  82 1186   43]
 [  15   66   63]]
              precision    recall  f1-score   support

        -1.0       0.56      0.49      0.52       245
         0.0       0.87      0.90      0.89      1311
         1.0       0.51      0.44      0.47       144

    accuracy                           0.81      1700
   macro avg       0.65      0.61      0.63      1700
weighted avg       0.80      0.81      0.80      1700

Precision:  0.6454845131274441
Recall:  0.612010162569986
F1:  0.6271832691348488
Accuracy:  0.8058823529411765


In [12]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7146668921961612

### random 15

In [16]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 121   97   18]
 [  91 1190   52]
 [  18   61   52]]
              precision    recall  f1-score   support

        -1.0       0.53      0.51      0.52       236
         0.0       0.88      0.89      0.89      1333
         1.0       0.43      0.40      0.41       131

    accuracy                           0.80      1700
   macro avg       0.61      0.60      0.61      1700
weighted avg       0.80      0.80      0.80      1700

Precision:  0.6117019274086263
Recall:  0.6007938700291582
F1:  0.6060363193089818
Accuracy:  0.8017647058823529


In [17]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7087993238207022

### random 25

In [28]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 119   78   17]
 [  78 1246   47]
 [  17   55   43]]
              precision    recall  f1-score   support

        -1.0       0.56      0.56      0.56       214
         0.0       0.90      0.91      0.91      1371
         1.0       0.40      0.37      0.39       115

    accuracy                           0.83      1700
   macro avg       0.62      0.61      0.62      1700
weighted avg       0.83      0.83      0.83      1700

Precision:  0.6204990749086768
Recall:  0.6129378281744695
F1:  0.6165479906414486
Accuracy:  0.8282352941176471


In [29]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7217082671699372

### random 35

In [38]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 130   73   17]
 [  64 1247   51]
 [  17   52   49]]
              precision    recall  f1-score   support

        -1.0       0.62      0.59      0.60       220
         0.0       0.91      0.92      0.91      1362
         1.0       0.42      0.42      0.42       118

    accuracy                           0.84      1700
   macro avg       0.65      0.64      0.64      1700
weighted avg       0.84      0.84      0.84      1700

Precision:  0.6479364303863772
Recall:  0.6405762244259967
F1:  0.6441620230031898
Accuracy:  0.8388235294117647


In [39]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7423654822056167

### random 45

In [43]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 112   68   14]
 [  81 1251   49]
 [  15   64   46]]
              precision    recall  f1-score   support

        -1.0       0.54      0.58      0.56       194
         0.0       0.90      0.91      0.91      1381
         1.0       0.42      0.37      0.39       125

    accuracy                           0.83      1700
   macro avg       0.62      0.62      0.62      1700
weighted avg       0.83      0.83      0.83      1700

Precision:  0.6216784005396714
Recall:  0.6170616342060015
F1:  0.6185287214403571
Accuracy:  0.8288235294117647


In [44]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.722274463208235